In [1]:
import numpy as np
from astropy.io import fits
from astropy.coordinates import Angle
import matplotlib.pyplot as plt
from scipy.stats import scoreatpercentile
from scipy import ndimage,signal
import glob
import os
import sys

from matplotlib.ticker import MultipleLocator
from mpl_toolkits.axes_grid1 import make_axes_locatable
import matplotlib.ticker as ticker

from ppxf.ppxf import ppxf, rebin
import ppxf.ppxf_util as util
from ppxf import miles_util as lib

import threading
from argparse import ArgumentParser

c = 299792.458 # speed of light in km/s

In [5]:
#### VARIABLES ####
root_in            = "/Users/00094926/repos/A3DRA/project_caro/simspin_files/cubes/"
root_out           = "/Users/00094926/repos/A3DRA/project_caro/simspin_files/cubes/ppxf_output/"
template_loc       = "/Users/00094926/repos/A3DRA/project_caro/EMILES_PADOVA_CH/Ech1.30*.fits"
file_name_list     = ["disk_EMILES_spectral_fwhm0_highz",
                      "bulge_EMILES_spectral_fwhm0_highz",
                      "old_bulge_EMILES_spectral_fwhm0_highz",
                      "disk_BC03hr_spectral_fwhm0_highz",
                      "bulge_BC03hr_spectral_fwhm0_highz",
                      "old_bulge_BC03hr_spectral_fwhm0_highz"]
folder_list        = ["jun16_2023/fwhm0_highz/disk_EMILES/",
                      "jun16_2023/fwhm0_highz/bulge_EMILES/",
                      "jun16_2023/fwhm0_highz/old_bulge_EMILES/",
                      "jun16_2023/fwhm0_highz/disk_BC03hr/",
                      "jun16_2023/fwhm0_highz/bulge_BC03hr/",
                      "jun16_2023/fwhm0_highz/old_bulge_BC03hr/"]
redshift           = 0.3
telescope_FWHM_list= [0, 0, 0, 0, 0, 0]
template_FWHM_list = [2.51, 2.51, 2.51, 3, 3, 3]
mom                = [4,4,4,2,2,2]
plot               = True


In [6]:
    i = 5
    
    folder = folder_list[i]
    template_FWHM = template_FWHM_list[i]
    file_name = file_name_list[i]
    telescope_FWHM = telescope_FWHM_list[i]
    FWHM_temp = 2.51
    z = redshift
    n_mom = mom[i]
        
    #### COMPUTATION based on variables ####
    if telescope_FWHM <= template_FWHM:
        FWHM_gal = (template_FWHM * (1 + z)) # Angstroms (resolution for this "observed" template)
    else:
        FWHM_gal = telescope_FWHM

    file_root = f"{root_in}{folder}"
    ppxf_out_root = f"{root_out}{folder}"
    #########################################

    inputFileExists = os.path.exists(f"{file_root}{file_name}.FITS")
    outputDirectoryExists = os.path.exists(ppxf_out_root)

    if not inputFileExists or not outputDirectoryExists:
        print("\nError - check you file names before proceading.")

        if not inputFileExists:
            print(f"Input FITS file (",f"{file_root}{file_name}.FITS", ") does not exist. \n")
            sys.exit()
        else:
            print("Output directory (",f"{ppxf_out_root}",") for pPXF files does not exist. \n")
            sys.exit()

    # Check n_mom is 2 or 4

    if n_mom != 2. and n_mom != 4.:
        print("Number of requested velocity moments must be 2 OR 4.")
        sys.exit()

    # Make a new pPXF fit folder at the location of ppxf_out_root for fitting plots
    try:
      os.makedirs(f"{ppxf_out_root}/pPXF_fits", exist_ok=True)
    except OSError as error:
      print("Invalid path name for output pPXF files. \n")
      sys.exit()
        
    #---------------------------------#
    # Step 1: Read in the observation #
    #---------------------------------#

    hdu = fits.open(f"{file_root}{file_name}.FITS")

    cube     = hdu["DATA"].data
    head_obs = hdu["DATA"].header
    var      = hdu["STAT"].data*1e40
    
    npart    = hdu["NPART"].data
    npart_flat = npart.reshape(-1)
    npart_flat[npart_flat == 0] = None

    hdu.close()
    
    print(f"Reading spectral cube of size {cube.shape}.")

    # Reshape the cube into an array
    npix = cube.shape[0]
    nspec = cube.shape[1] * cube.shape[2]
    spectrum = cube.reshape(npix, -1) # create an array of spectra [npix, (sbin_x * sbin_y)]
    noise_flat = var.reshape(npix, -1) 

    print(f"Re-shaping cube into array of shape {spectrum.shape}, with {spectrum.shape[0]} wavelengths and {spectrum.shape[1]} pixels.")
    print(f"Re-shaping noise into array of shape {noise_flat.shape}, with {noise_flat.shape[0]} wavelengths and {noise_flat.shape[1]} pixels.")

    # Begin by pulling out the wavelength axis of the cube -------------------------------------------------------------
    # These properties are necessary for logarithmically re-binning the spectra before
    # fitting with pPXF.
    wave_axis = head_obs['CRVAL3'] + (np.arange(0., (head_obs['NAXIS3']))*head_obs['CDELT3'])
    wave_range = head_obs["CRVAL3"] + np.array([0., head_obs["CDELT3"]*(head_obs["NAXIS3"]-1)])
    velscale = c * np.diff(np.log(wave_axis[-2:])) # Smallest velocity step
    velscale = velscale[0]

    print(f"The velocity scale of the `observed` spectrum is {velscale} km/s per pixel.")

    # We also need to bring the observed spectrum back to the rest-frame ------------------------------------
    if z >= 0.0:
        wave_range = wave_range / (1 + z)
        FWHM_gal = FWHM_gal / (1 + z)
        z = 0
        print(f"The wavelength range of the 'observed' galaxy (returned to rest-frame) is {wave_range} Angstrom.")
        print(f"The FWHM at this restframe is {FWHM_gal}.")

    else:
        print(f"The wavelength range of the 'observed' galaxy is {wave_range} Angstrom.")
        print(f"The FWHM of the observation is {FWHM_gal}.")

    #---------------------------------#
    # Step 2: Read in the templates   #
    #---------------------------------#
    # Next, load the templates necessary to perform the fit with pPXF

    # Using pPXF function to read in the templates located at `template_loc`, this
    # function reads each template spectrum into an array sorted by age and metallicity.
    # If the FWHM_gal is larger than the FWHM_temp, this function will also convolve
    # the templates down to the same resolution as the observations.

    print(f"The FWHM of the templates is {FWHM_temp}.")
    if FWHM_gal <= FWHM_temp:
        FWHM_gal = None
        FWHM_temp = None
        print(f"No convolution can be made. Setting FWHM_gal, FWHM_temp = None.")

    miles = lib.miles(pathname = template_loc, velscale = velscale, FWHM_gal = FWHM_gal, FWHM_tem = FWHM_temp, norm_range=[5000, 5950])

    templates = miles.templates
    stars_templates = templates.reshape(templates.shape[0], -1)
    stars_templates /= np.median(stars_templates) # Normalizes stellar templates by a scalar

    log_lam_temp = miles.ln_lam_temp
    #reg_dim = miles.templates.shape[1:]
    #regul_err = 0.013  # Desired regularization error

    mask = (np.exp(log_lam_temp) > (np.min(wave_range)-500)) & (np.exp(log_lam_temp) < (np.max(wave_range)+500))

    log_lam_temp = log_lam_temp[mask]
    stars_templates = stars_templates[mask]

    wave_range_temp =[np.exp(np.min(log_lam_temp)), np.exp(np.max(log_lam_temp))]

    print(f"The wavelength range of the template spectra is {wave_range_temp} Angstrom.")
    print(f"Array of templates containing shape {stars_templates.shape}.")

    #----------------------------------------#
    # Step 3: Fitting the observed spectra   #
    #----------------------------------------#

    # Preparing the galaxy observations for input into pPXF
    galaxy, log_lam_galaxy, velscale = util.log_rebin(wave_range, spectrum[:,11], velscale=velscale)
    galaxy /= np.median(galaxy)

    dv = (log_lam_temp[0] - log_lam_galaxy[0])*c
    vel = c*np.log(1 + z)
    start = [vel, 100.] # starting guess for kinematic measurements

    print(f"The starting guess velocity, as according to Cappellari 2017 [V = c*log(1 + z)] is {vel} km/s.")

    # Now fitting each pixel in turn and comparing to the LOS_VEL and LOS_DISP values ----------------------------------
    if n_mom == 2:
        ppxf_velocity    = np.empty_like(npart_flat)
        ppxf_dispersion  = np.empty_like(npart_flat)
        ppxf_chi2        = np.empty_like(npart_flat)
    else:
        ppxf_velocity    = np.empty_like(npart_flat)
        ppxf_dispersion  = np.empty_like(npart_flat)
        ppxf_h3          = np.empty_like(npart_flat)
        ppxf_h4          = np.empty_like(npart_flat)
        ppxf_chi2        = np.empty_like(npart_flat)

    print(f"The 'observed' spectrum has a staring wavelength of {log_lam_galaxy[0]} Angstrom.")
    print(f"The template spectrum has a staring wavelength of {log_lam_temp[0]} Angstrom.")
    print(f"The difference is therefore {dv} km/s.")

    for j in range(nspec):
        if npart_flat[j] > 0:
            print(f"Fitting pixel [{j}]...")
            galaxy, log_lam_galaxy, velscale = util.log_rebin(wave_range, spectrum[:,j], velscale=velscale)
            noise_rebin, log_lam_noise, velscale_2 = util.log_rebin(wave_range, 1/np.sqrt(noise_flat[:,j]), velscale=velscale)
            noise_rebin /= np.median(galaxy)
            galaxy /= np.median(galaxy)
            
            if n_mom == 2:
                pp = ppxf(stars_templates, galaxy, noise_rebin, velscale, start,
                          vsyst=dv, lam = np.exp(log_lam_galaxy),
                          plot=False, moments=2, degree=-1, mdegree=5,  # Set plot to False when using in batch mode. change mdegree=10 for testing
                          clean=False, regul=False)
                ppxf_velocity[j]   = pp.sol[0]
                ppxf_dispersion[j] = pp.sol[1]
                ppxf_chi2[j]       = pp.chi2

                if plot:
                  pp.plot()
                  plt.savefig(f"{ppxf_out_root}/pPXF_fits/pPXF_fit_{j}.png")
                  plt.close()

            else:
                pp = ppxf(stars_templates, galaxy, noise_rebin, velscale, start,
                          vsyst=dv, lam = np.exp(log_lam_galaxy), 
                          plot=False, moments=4, degree=-1, mdegree=5,  # Set plot to False when using in batch mode. change mdegree=10 for testing
                          clean=False, regul=False)
                
                ppxf_velocity[j]   = pp.sol[0]
                ppxf_dispersion[j] = pp.sol[1]
                ppxf_h3[j]         = pp.sol[2]
                ppxf_h4[j]         = pp.sol[3]
                ppxf_chi2[j]       = pp.chi2

                if plot:
                  pp.plot()
                  plt.savefig(f"{ppxf_out_root}/pPXF_fits/pPXF_fit_{j}.png")
                  plt.close()


        else:
            if n_mom == 2:
                print(f"Pixel [{j}] is empty, proceading to next one...")
                ppxf_velocity[j]   = np.nan
                ppxf_dispersion[j] = np.nan
                ppxf_chi2[j]       = np.nan

            else:
                print(f"Pixel [{j}] is empty, proceading to next one...")
                ppxf_velocity[j]   = np.nan
                ppxf_dispersion[j] = np.nan
                ppxf_h3[j]         = np.nan
                ppxf_h4[j]         = np.nan
                ppxf_chi2[j]       = np.nan

    # --------------------------------- #
    # Step 4: Reshaping the fit results #
    #         back to images            #
    # --------------------------------- #

    fit_velocity   = ppxf_velocity.reshape([cube.shape[1], cube.shape[2]])
    fit_dispersion = ppxf_dispersion.reshape([cube.shape[1], cube.shape[2]])
    fit_chi2       = ppxf_chi2.reshape([cube.shape[1], cube.shape[2]])

    np.save(f"{ppxf_out_root}{file_name}_ppxf_velocity.npy", fit_velocity)
    np.save(f"{ppxf_out_root}{file_name}_ppxf_dispersion.npy", fit_dispersion)
    np.save(f"{ppxf_out_root}{file_name}_ppxf_chi2.npy", fit_chi2)

    if n_mom == 4:

        fit_h3 = ppxf_h3.reshape([cube.shape[1], cube.shape[2]])
        fit_h4 = ppxf_h4.reshape([cube.shape[1], cube.shape[2]])
        np.save(f"{ppxf_out_root}{file_name}_ppxf_h3.npy", fit_h3)
        np.save(f"{ppxf_out_root}{file_name}_ppxf_h4.npy", fit_h4)
        
    print(f"Done with the {i} iteration.")
    del cube, galaxy, noise_rebin, var, head_obs


Reading spectral cube of size (667, 24, 24).
Re-shaping cube into array of shape (667, 576), with 667 wavelengths and 576 pixels.
Re-shaping noise into array of shape (667, 576), with 667 wavelengths and 576 pixels.
The velocity scale of the `observed` spectrum is 156.50872608139352 km/s per pixel.
The wavelength range of the 'observed' galaxy (returned to rest-frame) is [2884.61538462 4421.53846154] Angstrom.
The FWHM at this restframe is 3.0.
The FWHM of the templates is 2.51.
The wavelength range of the template spectra is [2385.0168563137318, 4919.953772007879] Angstrom.
Array of templates containing shape (1388, 258).
The starting guess velocity, as according to Cappellari 2017 [V = c*log(1 + z)] is 0.0 km/s.
The 'observed' spectrum has a staring wavelength of 7.967007802933656 Angstrom.
The template spectrum has a staring wavelength of 7.776961470934251 Angstrom.
The difference is therefore -56974.457003985815 km/s.
Pixel [0] is empty, proceading to next one...
Pixel [1] is empty

/var/folders/45/wfwdmf2d3fdc9_ffnwcjsgrhhqjvjp/T/ipykernel_61657/1862387165.py:138: RuntimeWarning: invalid value encountered in true_divide
  galaxy /= np.median(galaxy)


 Best Fit:       Vel     sigma
 comp.  0:       -33       212
chi2/DOF: 28.58; DOF: 812; degree = -1; mdegree = 5
method = capfit; Jac calls: 7; Func calls: 58; Status: 2
linear_method = lsq_box; Nonzero Templates (>0.1%): 3/258
Fitting pixel [54]...
 Best Fit:       Vel     sigma
 comp.  0:       -19         2
chi2/DOF: 53.99; DOF: 812; degree = -1; mdegree = 5
method = capfit; Jac calls: 6; Func calls: 50; Status: 2
linear_method = lsq_box; Nonzero Templates (>0.1%): 3/258
Fitting pixel [55]...
 Best Fit:       Vel     sigma
 comp.  0:         2       181
chi2/DOF: 18.74; DOF: 812; degree = -1; mdegree = 5
method = capfit; Jac calls: 6; Func calls: 52; Status: 2
linear_method = lsq_box; Nonzero Templates (>0.1%): 3/258
Fitting pixel [56]...
 Best Fit:       Vel     sigma
 comp.  0:       -87       711
chi2/DOF: 49.02; DOF: 812; degree = -1; mdegree = 5
method = capfit; Jac calls: 17; Func calls: 144; Status: 2
linear_method = lsq_box; Nonzero Templates (>0.1%): 4/258
Fitting pixel [5

KeyboardInterrupt: 